In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import pickle

In [2]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
file = 'rete.pickle'
with open(path + file, 'rb') as f:
    G_real = pickle.load(f)

In [5]:
gradi_nodi_dict = dict(G_real.degree())
degree_distribution = {}
for degree in gradi_nodi_dict.values():
    if degree in degree_distribution:
        degree_distribution[degree] += 1
    else:
        degree_distribution[degree] = 1


In [6]:
sorted_degree_distribution = sorted(degree_distribution.items(), key=lambda x: x[1], reverse=True)
degrees, frequencies = zip(*sorted_degree_distribution)

In [7]:
def generate_random_graphs(N, p, num_graphs):
    
    degree_distributions = []
    diameters = []
    n_connected_components = []
    max_connected_component_sizes = []
    n_edges = []
    clustering_coefficients = []
    pearson_values = []
    avg_shortest_paths = []
    
    for _ in range(num_graphs):
        G = nx.erdos_renyi_graph(N, p)
        connected_components = list(nx.connected_components(G))
        degrees = [degree for node, degree in G.degree()]
        
        degree_distributions.append(degrees)
        n_connected_components.append(len(connected_components))
        n_edges.append(G.number_of_edges())
        max_connected_component_sizes.append(max([len(cc) for cc in connected_components]))
        clustering_coefficients.append(nx.average_clustering(G)) 
        avg_shortest_paths.append(nx.average_shortest_path_length(G))   

        gradi_nodi_dict = dict(G.degree())
        gradi_origine = []
        gradi_destinazione = []
        for u, v in G.edges():
            gradi_origine.append(gradi_nodi_dict[u])
            gradi_destinazione.append(gradi_nodi_dict[v])
        coeff_pearson = np.corrcoef(gradi_origine, gradi_destinazione)[0, 1]
        pearson_values.append(coeff_pearson)

        components_diameters = []
        
        for component in connected_components:
            components_diameters.append(nx.diameter(G.subgraph(component)))
        diameters.append(max(components_diameters))
        
    return degree_distributions \
        ,(sum(diameters)/num_graphs) \
        ,(sum(n_connected_components)/num_graphs) \
        ,(sum(n_edges)/num_graphs) \
        ,(sum(max_connected_component_sizes)/num_graphs) \
        ,(sum(clustering_coefficients)/num_graphs) \
        ,(sum(pearson_values)/(num_graphs)) \
        ,(sum(avg_shortest_paths)/num_graphs)

In [8]:
def plot_average_degree_distribution(degree_distributions, N, p):
    # Calcola la distribuzione media del grado
    max_degree = max(max(dist) for dist in degree_distributions)
    average_distribution = np.zeros(max_degree + 1)
    for dist in degree_distributions:
        for degree in dist:
            average_distribution[degree] += 1
    
    average_distribution /= len(degree_distributions)
    
    # Normalizza rispetto al numero totale di nodi
    average_distribution /= N
    #plt.bar(range(len(average_distribution)), average_distribution, color='b', alpha=0.7)
    #plt.title(f"Distribuzione Media del Grado\nN={N}, p={p}")
    #plt.xlabel("Grado")
    #plt.ylabel("Frequenza Media (Normalizzata)")
    #plt.show()
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=list(range(len(average_distribution)+1)),
        y=average_distribution,
        name=f"Distribuzione Media del Grado\nN={N}, p={p}",
        orientation='v',
        text=[round(ad,3) for ad in average_distribution],
        textposition='outside', 
        marker=dict(
            color='rgba(246, 78, 139, 0.6)',
            line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
        )
    ))
    fig.update_layout(
        title=dict(
            text=f"<b>Distribuzione Media del Grado\nN={N}, p={p}",  # Title text
            x=0.5,  # x=0.5 centers the title
            xanchor='center',  # 'center' ensures the title is centered
            font=dict(
                size=24,
                color='black',
                family="Arial, sans-serif",
            )
            ),
        xaxis=dict(
        title='Grado',
        tickmode='array',
        tickvals=list(range(len(average_distribution)+1)),  # Posizioni delle etichette sull'asse Y
        ticktext=list(range(len(average_distribution)+1)),  # Testo delle etichette sull'asse Y
        ),
        yaxis=dict(
             title="Frequenza Media (Normalizzata)"
        )
    )
    fig.update_layout(
        width=1000,   # Larghezza in pixel
        height=600   # Altezza in pixel
    )
    fig.show()

In [9]:
# Parametri
N = 22  # Numero di nodi
p = 0.53  # Probabilità di collegamento
num_graphs = 10000  # Numero di grafi casuali da generare

In [10]:
degree_distributions, avg_diameter, avg_n_connected_components, avg_n_edges, avg_max_cc_size, avg_clustering, avg_pearson, avg_sh_path = generate_random_graphs(N, p, num_graphs)
print("Diametro medio: ",avg_diameter)
print("numero medio di componenti connesse: ", avg_n_connected_components)
print("numero medio di archi: ",avg_n_edges)
print("numero medio della dimensione delle componenti connesse: ", avg_max_cc_size)
print("clustering medio", avg_clustering)
print("Pearosn medio: ", avg_pearson)
print("Densità media: ", ((2*avg_n_edges)/(N*(N-1))))
print("Average shortest path: ", avg_sh_path)
plot_average_degree_distribution(degree_distributions, N, p)

Diametro medio:  2.1129
numero medio di componenti connesse:  1.0
numero medio di archi:  122.4291
numero medio della dimensione delle componenti connesse:  22.0
clustering medio 0.5300343532426085
Pearosn medio:  -0.07551015382314817
Densità media:  0.529996103896104
Average shortest path:  1.4706480519480618


In [19]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=[freq/22 for freq in frequencies],
    x=degrees,
    name='distribuzione del grado',
    orientation='v',
    text=[round(freq/22,2) for freq in frequencies],
    textposition='outside', 
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.update_layout(
    title=dict(
        text='<b>Distribuzione di frequenza del grado dei nodi',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=24,
            color='black',
            family="Arial, sans-serif",
        )
        ),
    xaxis=dict(
    title='Gradi',
    tickmode='array',
    tickvals=list(range(min(degrees),max(degrees)+1)),  # Posizioni delle etichette sull'asse Y
    ticktext=list(range(min(degrees),max(degrees)+1)),  # Testo delle etichette sull'asse Y
    ),
    yaxis=dict(
         title='Frequenze'
    )
)
fig.update_layout(
    width=1000,   # Larghezza in pixel
    height=600   # Altezza in pixel
)

In [28]:
deg_dist = {}
for i in list(range(len(average_distribution)+1)):
    if i in list(dict(sorted_degree_distribution).keys()):
        deg_dist[i] = dict(sorted_degree_distribution)[i]/N
    else:
        deg_dist[i] = 0

In [62]:
fig = go.Figure()
testo_marker = list(range(len(average_distribution)+1))
testo_marker = [str(n) for n in testo_marker]
max_degree = max(max(dist) for dist in degree_distributions)
average_distribution = np.zeros(max_degree + 1)
for dist in degree_distributions:
    for degree in dist:
        average_distribution[degree] += 1
average_distribution /= len(degree_distributions)
average_distribution /= N
fig.add_trace(go.Scatter(
    x=list(range(len(average_distribution)+1)), 
    y=average_distribution, mode='lines+markers', 
    line=dict(color='rgb(255, 0, 0)'),
    text=testo_marker, 
    textposition='top center',
    name=f'Random Graph'))
fig.add_trace(go.Scatter(
    x=list(range(len(average_distribution)-3)),
    y=list(deg_dist.values()),
    mode='lines+markers', 
    line=dict(color='rgb(0, 0, 255)'), 
    name=f'Real Network',
    text=testo_marker, 
    textposition='top center'))
fig.update_layout(    title=dict(
    text='<b>P of a node n having grade k',  # Title text
    x=0.5,  # x=0.5 centers the title
    xanchor='center',  # 'center' ensures the title is centered
    font=dict(
        size=24,
        color='black',
        family="Arial, sans-serif",
    )
    ),
                xaxis_title='Degree',
                yaxis=dict(
                    title='Probability of a node to have k edges'
                ),
                width=800,   # Larghezza in pixel
                height=600 )

fig.show()